# Notebook for calculating the calcium event probability - Part 1

This use this jupyter notebook for the analysis of your in vitro imaging data (best suited for cell culture data). Before you run this notebook there are a few steps you need to do first:

1. Load your raw imaging tiffs into imagej/fiji
2. Draw the ROIs **IMPORTANT**: Last ROI must be a background ROI 
3. Calculate mean grey value with imagej/fiji 
4. Export your results as csv file



In [ ]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

from os import listdir
from os.path import isfile, join
import pingouin as pg

# Pre processing function:
def pre_processing(df):
    background=df[df.columns[-1]].tolist()  #select background column and make list
    df=df.drop(df.columns[-1], axis=1) #delete last column (background)
    df=df.drop(df.columns[0], axis=1) #delte first column (frame)
    df_background_sub=df.sub(background, axis=0) #subtract background from every other column
    
    return df_background_sub

def dff_calc(df_input,f_base_value=0,dff_median=False):
    
    for column_name in df_input:
        column=df_input[column_name]
        column=column.astype('float')
        if dff_median:
            median_value=column.median()
            dff_series=column.apply((lambda x: (x - median_value)/median_value))
            df_input[column_name]=dff_series
        else:
            dff_series=column.apply((lambda x: (x - column.iloc[f_base_value])/column.iloc[f_base_value])) 
            df_input[column_name]=dff_series
        
    return df_input


#pre_processing_bool=True
frames_to_seconds=True
fps_rate=5 

## Defining some parameters for your analysis

In the next cell define some parameter:

* Do you want to perform preprocessing (i.e. background substraction) of your imagej/fiji data? 
 * default: `pre_processing_bool=True` <br>
<br>
* Do you want to exclude specific ROIs?         
 * default: `exclude_rois=True` <br>
<br>    
* Do you want to convert frames to seconds? If so input framerate in fps.
 * default: `frames_to_seconds=True` 
 * default: `fps_rate=1` <br>
<br>

**Next**: Input the path to your results csv file from imagej/fiji input the variable: path_to_data

In [ ]:
pre_processing_bool=True
frames_to_seconds=True
fps_rate=5 

path_to_data=r'C:\Users\Lutz\OneDrive\Desktop\Data Science\calcium event probability\Calcium_Events\Calcium_Events\8OHDPAT\20220118_8OH_Slice2_allCells'

file_extension='.csv'

if pre_processing_bool==True:
    df=pre_processing(pd.read_csv(path_to_data + file_extension))
else:
    df=pd.read_csv(path_to_data + file_extension)

if frames_to_seconds:
    df.index=df.index/fps_rate
    x_label='Time (s)'
else:
    x_label='Frames'


## Calculation of $\Delta$f/f

Now you can calculate $\Delta$f/f but first define the parameters for that: 
* If you want to calculate $\Delta$f/f with the first frame as your f0, you do not need to change anything.
* If you want to calculate $\Delta$f/f with the median fluorescence as your f0, set the parameter `dff_median=True` <br> (i.e. `dff_frame=dff_calc(df_copy, dff_median=True)`)

It also plots each of your ROIs in a subplot and also all ROIs combined
    



In [ ]:
df_copy=df.copy()
dff_frame=dff_calc(df_copy, dff_median=True)     

In [ ]:
#plotting individual ROIs in individual subplots
col_num=len(dff_frame.columns)
number_of_subplots=list(range(0,len(dff_frame.columns)))
axes=dff_frame.plot(subplots=True, layout=(12, 5), figsize=(24, 24), sharex=True,sharey=True, legend=None, title=number_of_subplots)

for c in axes:
   for ax in c:
      ax.axhline(y=0, color='k', ls='dashed')

plt.show()

#plotting individual ROIs in ONE plot (overlayed)
dff_frame.plot(legend=None)
ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

## Excluding ROIs (optional)

If you want exclude some of the ROIs from above set `exclude_rois=True` (default: `exclude_rois=False`) and put the numbers of the ROIs you want to exclude into the list and seperate every ROI by a "," (`list_of_excluded_rois`) below. **IMPORTANT**: Start counting from 0!

It also saves the a new csv file into the current path with the addtion of *_cleaned*

In [ ]:
exclude_rois=False
list_of_excluded_rois=[25] #for the demo

if exclude_rois:
    cleaned_df=dff_frame.copy()
    exclude_columns=cleaned_df.iloc[:, list_of_excluded_rois].columns
    cleaned_df=cleaned_df.drop(columns=exclude_columns)    
    cleaned_df.to_csv((path_to_data + '_cleaned' + file_extension), index=False)  
    
    df_mean=cleaned_df.mean(axis=1)
    df_std=cleaned_df.std(axis=1)
    df_sem=cleaned_df.sem(axis=1)
    
else:
    cleaned_df=dff_frame.copy()
    cleaned_df.to_csv((path_to_data + '_cleaned' + file_extension), index=False)
    
    df_mean=dff_frame.mean(axis=1)
    df_std=dff_frame.std(axis=1)
    df_sem=dff_frame.sem(axis=1)

Now plot the ROIs again to make sure you deleted the correct ROIs

In [ ]:
cleaned_df

In [ ]:
#plotting individual ROIs in individual subplots
if exclude_rois:
    
    col_num=len(cleaned_df.columns)
    number_of_subplots=list(range(0,len(cleaned_df.columns)))
    axes=cleaned_df.plot(subplots=True, layout=(12, 5), figsize=(24, 24), sharex=True,sharey=True, legend=None, title=number_of_subplots)

    for c in axes:
       for ax in c:
          ax.axhline(y=0, color='k', ls='dashed')

    plt.show()

    #plotting individual ROIs in ONE plot (overlayed)
    cleaned_df.plot(legend=None)
    ax = plt.gca()
    ax.axhline(y=0, color='k', ls='dashed')
    plt.xlabel(x_label)
    plt.ylabel('\u0394 F/F')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.show()

## Plotting of the mean time courses

Now you can start plotting the mean of all your ROIs. But first define if you want to save the images (default: `save_images=False`)

In [ ]:
save_images=False

#plotting mean + SD

plt.plot(df_mean, color='black', linewidth=0.75)
plt.fill_between(df_mean.index, df_mean-df_std, df_mean+df_std, alpha=0.5)

ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
plt.title('Mean + SD')
if save_images:
    plt.savefig((path_to_data[:-4] + '_mean_SD' + '.svg'), format = 'svg', dpi=300) # uncommand this if you want to save it as vector graphic
plt.show()
